In [1]:
%pip install scipy plotly pandas
from math import ceil
from scipy.optimize import minimize

Declaracion de variables

In [2]:
# DATOS GENERALES
total_campaña = 25000 # Kg
total_aceite = 18000 # L

# PERSONAL
capacidad_recogida = 495 # Kg/persona-dia
sueldo = 120 # €/persona-dia
alojamiento = 96 # €/dia-habitacion 
manutencion = 54 # €/persona-dia

# GASTOS FIJOS
gastos_fijos = 2600 # €/dia

# INSTALACIONES DEL TRUJAL
capacidad_prensa = 1200 # Kg/dia
capacidad_embotelladora = 1920 # L/dia

# EXTRAS
num_prensas = 1
num_embotelladoras = 1
max_prensas = 5
max_embotelladoras = 6
prensa_adicional = 8000 # €/campaña
embotelladora_adicional = 9600 # €/campaña
personas_maquina = 2
max_personas = 10

In [3]:
# ## ANDER
# # DATOS GENERALES
# total_campaña = 30000 # Kg
# total_aceite = 21600 # L

# # PERSONAL
# capacidad_recogida = 495 # Kg/persona-dia
# sueldo = 100 # €/persona-dia
# alojamiento = 80 # €/dia-habitacion 
# manutencion = 45 # €/persona-dia

# # GASTOS FIJOS
# gastos_fijos = 1800 # €/dia

# # INSTALACIONES DEL TRUJAL
# capacidad_prensa = 1350 # Kg/dia
# capacidad_embotelladora = 2160 # L/dia

# # EXTRAS
# num_prensas = 1
# num_embotelladoras = 1
# max_prensas = 5
# max_embotelladoras = 3
# prensa_adicional = 11000 # €/campaña
# embotelladora_adicional = 13200 # €/campaña
# personas_maquina = 2
# max_personas = 10

Funcion a optimizar

In [4]:
def opt_func(x: list[int]) -> tuple[float, float]: # (coste, plazo)
    personas = int(x[0])
    prensas = int(x[1])
    embotelladoras = int(x[2])

    # RECOGIDA
    dias_recogida = ceil(total_campaña / (personas * capacidad_recogida))

    # PRENSADO
    dias_prensado = ceil(total_campaña / ((1 + prensas) * capacidad_prensa))

    # EMBOTELLADO
    dias_embotellado = ceil(total_aceite / ((1 + embotelladoras) * capacidad_embotelladora))

    plazo = dias_recogida + dias_prensado + dias_embotellado
    coste_maquinas = prensas * prensa_adicional + embotelladoras * embotelladora_adicional
    gasto_personal =  plazo * (sueldo * personas + ceil(personas / 2) * alojamiento + manutencion * personas)
    coste = gasto_personal + coste_maquinas + gastos_fijos * plazo

    return (coste, plazo)


Limitaciones

In [5]:
# Constraints
def const1(x: list[int]) -> float:
    return max_personas - x[0]

def const2(x: list[int]) -> float:
    return max_prensas - (x[1] + 1) 

def const3(x: list[int]) -> float:
    return max_embotelladoras - (x[2] + 1)

def const4(x: list[int]) -> float:
    if x[1] + 1 == max_prensas and x[0] >= (x[1] + 1) * personas_maquina:
        return 0
    if x[0] > ((x[1] + 1) * personas_maquina) + (personas_maquina - 1) or x[0] < (x[1] + 1) * personas_maquina:
        return -1
    
    return 0

def const5(x: list[int]) -> float:
    if x[2] + 1 == max_embotelladoras and x[0] >= (x[2] + 1) * personas_maquina:
        return 0
    if x[0] > ((x[2] + 1) * personas_maquina) + (personas_maquina - 1) or x[0] < (x[2] + 1) * personas_maquina:
        return -1
    
    return 0

con1: dict = {'type': 'ineq', 'fun': const1}
con2: dict = {'type': 'ineq', 'fun': const2}
con3: dict = {'type': 'ineq', 'fun': const3}
con4: dict = {'type': 'eq', 'fun': const4}
con5: dict = {'type': 'eq', 'fun': const5}

cons: list[dict] = [con1, con2, con3, con4, con5]

Optimizacion y resultados

In [6]:
def check_consts(consts: list[dict], x) -> bool:
    for const in consts:
        if const['type'] == 'eq':
            if const['fun'](x) != 0:
                return False
        elif const['type'] == 'ineq':
            if const['fun'](x) < 0:
                return False
    return True

In [7]:
# Busqueda mediante fuerza bruta

min_coste = float("inf")
plazo = 0
personas = 0
prensas = 0
embotelladoras = 0
data = []
for i in range(1, max_personas + 1):
    for j in range(0, max_prensas + 1):
        for k in range(0, max_embotelladoras + 1):
            x = [i, j , k]
            if check_consts(cons, x):
                (c, p) = opt_func(x)
                data.append([i, j, k, c, p])
                if c < min_coste or (c == min_coste and p < plazo):
                    min_coste = c
                    plazo = p
                    personas = i
                    prensas = j
                    embotelladoras = k
            

print("Se relizará en " + str(plazo) + " dias costando " + str(min_coste) + " €")
print("Se contratarán " + str(personas) + " personas y se utilizarán " + str(prensas) + " prensas extra y " + str(embotelladoras) + " embotelladoras extra")


Se relizará en 20 dias costando 113840 €
Se contratarán 6 personas y se utilizarán 2 prensas extra y 2 embotelladoras extra


In [8]:
import plotly.express as px
import pandas as pd


In [9]:
df = pd.DataFrame(data, columns=["Personas", "Prensas", "Embotelladoras", "Coste", "Plazo"])
df

,Personas,Prensas,Embotelladoras,Coste,Plazo
0,2,0,0,173508,57
1,3,0,0,159072,48
2,4,1,1,118752,29
3,5,1,1,119066,27
4,6,2,2,113840,20
5,7,2,2,115038,19
6,8,3,3,122816,16
7,9,3,3,122490,15
8,10,4,4,133060,13


In [10]:
fig = px.scatter_3d(df, x='Personas', y='Prensas', z='Embotelladoras', color='Coste')
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed